## Use lib

In [72]:
import pandas as pd
import numpy as np
import copy
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
import nltk
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import re, joblib
import konlpy
from konlpy.tag import Okt 
okt = Okt()
import string
import threading
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import time

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()



## Use Data

### 연결된 구글드라이브 로 경로 바꾸기(windows)

In [2]:
os.chdir('G:\내 드라이브\제주_전기차\data')

### data list

In [11]:
df_blog = pd.read_csv('전기차감성분석(0325).csv')
df_cafe = pd.read_csv('카페통합감성추가.csv')
df_toflux = pd.read_csv('전기차전처리추가.csv')

C:\Users\alsdn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [63]:
df_sns = pd.read_csv('인스타전기차.csv')
df_news = pd.read_csv('CFI전기차보도자료.csv')

### data 확인

In [83]:
# print (df_blog.isnull().sum())
# print (df_cafe.isnull().sum())
print (df_toflux.isnull().sum())
# print (df_sns.isnull().sum())
# print (df_news.isnull().sum())

Unnamed: 0        394820
classCode              0
필터명               394820
수집채널                   0
카페명                23962
제목                     0
작성자                    0
작성시간                   0
조회수                    0
댓글수                    0
호감수                    0
비호감수                   0
공유수                    0
모델명               378979
별점                 40260
평가                   882
해시태그              394820
Url                    0
Contents               0
ChannelCode            0
ContentType            0
PostNo                 0
Sno                    0
Og_ImageUrl       394820
Og_Description    394820
grpNo                  0
SeqIdx                 0
전처리                   42
dtype: int64


In [82]:
df_toflux['전처리'].dropna(inplace=True)

In [6]:
df_sns.head(2)

,컨텐츠,날짜,좋아요,장소,태그
0,hyoseong_90신형 G90 첫출고 🥳사전계약하고도 오래걸렸지만기다려 주셔서 감...,2022-03-22,0,NaN,[]
1,ony._.chuu제주에서 바다 정말 많이가봣지만 개인적으로 가장 아름다운바다는 서...,2020-10-08,0,우도,"['#워니의✈️🌏', '#워니_jeju', '#제주', '#jeju', '#우도수정됨']"


In [15]:
df_toflux.columns


Index(['Unnamed: 0', 'classCode', '필터명', '수집채널', '카페명', '제목', '작성자', '작성시간',
       '조회수', '댓글수', '호감수', '비호감수', '공유수', '모델명', '별점', '평가', '해시태그', 'Url',
       'Contents', 'ChannelCode', 'ContentType', 'PostNo', 'Sno',
       'Og_ImageUrl', 'Og_Description', 'grpNo', 'SeqIdx', '전처리'],
      dtype='object')

In [84]:
df_toflux= pd.DataFrame(
    {
    '제목':df_toflux.제목,
    '내용':df_toflux.Contents,
    '작성시간':df_toflux.작성시간,
    '카페명':df_toflux.카페명,
    '평가':df_toflux.평가,
    '전처리':df_toflux.전처리,
    }    
)

In [97]:
df_toflux['카페명'].fillna('전기차동호회',inplace=True)
df_toflux['평가'].fillna('E',inplace=True)
df_toflux.dropna(inplace=True)

In [98]:
df_toflux['평가'].value_counts()

P    201518
E    103576
N     89684
Name: 평가, dtype: int64

In [99]:
df_toflux.isnull().sum()

제목      0
내용      0
작성시간    0
카페명     0
평가      0
전처리     0
dtype: int64

In [100]:
df_toflux.head(2)

,제목,내용,작성시간,카페명,평가,전처리
0,아이오닉5 휠베이스 3000의 가치,▶ 아이오닉5 휠베이스 3000의 가치_x000D_\n_x000D_\n서비스센터 갔...,2021-01-12 17:01:00,전기차동호회,P,아이오 닉 휠 베이스 가치 서비스 센터 가다 부품 수급 문제 로 렌터카 받다 오다 ...
1,"현대차 아이오닉5 vs 테슬라 모델Y, 뭘 살까요?","▶ 현대차 아이오닉5 vs 테슬라 모델Y, 뭘 살까요?_x000D_\n_x000D_...",2021-02-23 21:36:00,전기차동호회,P,현대차 아이오 닉 테슬라 모델 뭘 살다 요 현대차 아이오 닉 테슬라 모델 뭘 살다 ...


In [21]:
df_blog.head(2)

,제목,날짜,내용,긍부정,href,year,lr긍부정,lr긍부정이진분류,감성사전긍부정,전기차문장,전기차본문감성분석
0,2022년 4박 5일 신혼여행 후기 ing 제주도 (Feat. 일오반식당...,2022.03.17,제주도신혼여행후기내돈내산년 년 월 일 드뎌 결혼식이 끝나고 우도 가자마자 전기차를 ...,0,https://blog.naver.com/apple_378?Redirect=Log&...,2022,E,P,E,​ 오랜만에 바다를 보니 뻥 뚫리고 좋았어여 ㅎㅎ 같이 커플 트레이닝복으로 맞추고...,P
1,"제주공항숙소 시리우스 제주호텔: 트윈룸, 디럭스룸 장단점",2022.03.17,디럭스룸 후기 제주특별자치도 제주시 도령로 전화 공항의 지하 주차장은 지하층까지...,-1,https://blog.naver.com/zoonee96?Redirect=Log&l...,2022,E,P,F,​ 공항의 지하 주차장은 지하4층까지 있고 지하 1층에 전기차 충전소가 있어요 주차...,N


In [20]:
df_cafe.head(2)

,제목,내용,댓글,유형,조회수,작성날짜,월별,카페명,년별,lr긍부정,lr긍부정이진
0,봉고3더블캡,보다 투명하고 빠른 거래를 위하여 아래의 글 양식을 준수하여 주세요 거래 지역...,가격이 궁금해요..^^\n얼마인가요ㅡ\n,제사모 중고장터,130,2022.03.16.,2022.03,제사모,2022,P,P
1,렌트카 알아보고 있는데 lpg 차량이 저렴한 이유가 뭘까요?,육지에서 이번주 토욜부터 제주 한달살이 시작하면서 렌트카 알아보고 있어요 육지에선 ...,요즘 전기차가 대세기도 하지만 가스값 장난아니게 올랏어요...또한 한달살기라면 자차...,Q. 종합 질문방,161,2022.03.16.,2022.03,제사모,2022,N,N


In [33]:
df_cafe.columns

Index(['제목', '내용', '댓글', '유형', '조회수', '작성날짜', '월별', '카페명', '년별', 'lr긍부정',
       'lr긍부정이진'],
      dtype='object')

In [117]:
df_cafe_commet =pd.DataFrame(
    {
    '제목':df_cafe.제목,
    '내용':df_cafe.댓글,
    '작성날짜':df_cafe.작성날짜,
    '카페명':df_cafe.카페명,
    '평가':df_cafe.lr긍부정,
    '월별':df_cafe.월별,
    '년별':df_cafe.년별,
    }    
)

In [122]:
df_cafe_commet['내용'] = df_cafe_commet['내용'].apply(lambda x : None if x in ['내용없음','댓글이 없습니다','\n'] else x)

In [123]:
df_cafe_commet.내용.value_counts()

이야~~사진 너무예쁘네요^^\n낼출발하는데 기대많이하고하겠습니당당\n서쪽이 진리였네요ㅠㅠ\n동쪽에 가까워지니 눈길이 많아요ㅋ\n꽁닥네용작흑ㅡㅜ\n눈길운전조심하세요~\n삭제된 댓글입니다.\n헉 저흰 나와서 광치기해변 가고 있어요ㅋㅋ\n만날뻔 했네요ㅠㅠ\n햇살 비치는 오징어 자태가 넘 이쁜데요 ㅎㅎ\n춘미향에서도 꼭 한끼 먹어봐야겠다는~^^*\nㅎㅎ꼭 점심에 가셔요\n2시에 주문 마감입니다^^\n슈퍼그레잇 고기정식 정보부탁드려요^^\n춘미향이요^^\n저도 널린 오징어 득템하고 춘미향 들러봐야 겠어요~~ 사진 너무 좋네요\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [126]:
df_cafe_commet.dropna(inplace=True)

In [127]:
df_cafe_commet

,제목,내용,작성날짜,카페명,평가,월별,년별
0,봉고3더블캡,가격이 궁금해요..^^\n얼마인가요ㅡ\n,2022.03.16.,제사모,P,2022.03,2022
1,렌트카 알아보고 있는데 lpg 차량이 저렴한 이유가 뭘까요?,요즘 전기차가 대세기도 하지만 가스값 장난아니게 올랏어요...또한 한달살기라면 자차...,2022.03.16.,제사모,N,2022.03,2022
5,여행 고수님들 추천좀 해주세요,렌트카는 가스 예요.\n평소 본인이 운전하기 편한차로 하세요.\n74시간이면 2박3...,2022.03.14.,제사모,P,2022.03,2022
8,전기차 신차급,팔렸나요?\n,2022.03.10.,제사모,E,2022.03,2022
12,제주공항 전기차 주차요금..,승용차는 모두 소형입니다.\n저공해차량 50% 할인.\n공항에\n소형 대형 분류는\...,2022.03.05.,제사모,E,2022.03,2022
...,...,...,...,...,...,...,...
5909,제주도 전기차 도민 공모,보급대수 규모가 차원이 틀리네요..ㅎㅎ\n,2015.03.17.,전기차동호회,E,2015.03,2015
5910,[링크스크랩] 100% 순수 전기차 ‘SM3 Z.E.’ 보급 위해 특별 할인 제공,정부 보조금 까지 받게 된다면 엄청 싸게 살수 있겠네요....서울에서 다시 보조금 ...,2015.03.08.,전기차동호회,E,2015.03,2015
5911,"[링크스크랩] 자동차 업계, 제주도 전기차 민간 보급 공모 총력전…가격 인하 이어져",쏘울 내일 받는데 100만원 낮아지다니....ㅠㅠ\n서울도 다시한번더 보조금 지원하...,2015.03.04.,전기차동호회,E,2015.03,2015
5912,등업부탁해요 하반기 전기차를 신청하고싶었는데,반가워요~~😊\n내년에 제주도 전기차 공급량이 대박이던데 좋은소식 있으시길...\n...,2014.10.26.,전기차동호회,P,2014.10,2014


#### 댓글칼럼 토큰화 및 머신러닝

In [128]:
df_cafe_commet['token']=df_cafe_commet.내용.apply(lambda x : re.sub('[^가-힣]',' ',x))

In [129]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 자 와 ㅋㅋ ㅠㅠ ㅎㅎ'
stopwords = stop_text.split()
X_val = []
for sentence in tqdm(df_cafe_commet.token):
    morphs = okt.morphs(sentence, stem=True)
    tmp_list = [word for word in morphs if word not in stopwords]
    tmp_str = ' '.join(tmp_list)
    X_val.append(tmp_str)

100%|██████████| 5079/5079 [01:58<00:00, 42.76it/s] 


In [130]:
df_cafe_commet['token'] = X_val

In [73]:
la = LabelEncoder()
la.fit(df_toflux.평가)

LabelEncoder()

In [101]:
X_train,X_test,y_train,y_test = train_test_split(
    df_toflux.전처리,df_toflux.평가,stratify=df_toflux.평가,random_state=2022
)

In [102]:
y_train_la = la.transform(y_train)
y_test_la = la.transform(y_test)

In [103]:

cvect = CountVectorizer()
cvect.fit(X_train)

CountVectorizer()

In [131]:
X_train_cv=cvect.transform(X_train)
X_test_cv=cvect.transform(X_test)
X_val_cv=cvect.transform(X_val)

In [105]:
lr = LogisticRegression(max_iter=350)
lr.fit(X_train_cv,y_train_la)

C:\Users\alsdn\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=350)

In [107]:
lr.score(X_test_cv,y_test_la)

0.8147525203911039

In [132]:
la.inverse_transform(lr.predict(X_val_cv))

array(['E', 'P', 'P', ..., 'P', 'P', 'E'], dtype=object)

In [133]:
df_cafe_commet['평가']=la.inverse_transform(lr.predict(X_val_cv))

In [134]:
df_cafe_commet.head(2)

,제목,내용,작성날짜,카페명,평가,월별,년별,token
0,봉고3더블캡,가격이 궁금해요..^^\n얼마인가요ㅡ\n,2022.03.16.,제사모,E,2022.03,2022,가격 궁금하다 얼마 인가요
1,렌트카 알아보고 있는데 lpg 차량이 저렴한 이유가 뭘까요?,요즘 전기차가 대세기도 하지만 가스값 장난아니게 올랏어요...또한 한달살기라면 자차...,2022.03.16.,제사모,P,2022.03,2022,요즘 전기차 대세 기도 하지만 가스 값 장난 아니다 올 랏 어 요 또한 한 달 살기...


In [135]:
df_cafe_commet.to_csv('cafe댓글추출후분석.csv',index=False)

In [142]:
df_cafe['내용'] = df_cafe['내용'].apply(lambda x : None if x in ['내용없음','내용이 없습니다','\n'] else x)

In [144]:
df_cafe.dropna(inplace=True)

In [146]:
df_cafe.isnull().sum()

제목         0
내용         0
댓글         0
유형         0
조회수        0
작성날짜       0
월별         0
카페명        0
년별         0
lr긍부정      0
lr긍부정이진    0
dtype: int64

In [148]:
df_cafe_commet.head(2)

,제목,내용,작성날짜,카페명,평가,월별,년별,token
0,봉고3더블캡,가격이 궁금해요..^^\n얼마인가요ㅡ\n,2022.03.16.,제사모,E,2022.03,2022,가격 궁금하다 얼마 인가요
1,렌트카 알아보고 있는데 lpg 차량이 저렴한 이유가 뭘까요?,요즘 전기차가 대세기도 하지만 가스값 장난아니게 올랏어요...또한 한달살기라면 자차...,2022.03.16.,제사모,P,2022.03,2022,요즘 전기차 대세 기도 하지만 가스 값 장난 아니다 올 랏 어 요 또한 한 달 살기...


In [147]:
df_cafe.head.head(2)

,제목,내용,댓글,유형,조회수,작성날짜,월별,카페명,년별,lr긍부정,lr긍부정이진
0,봉고3더블캡,보다 투명하고 빠른 거래를 위하여 아래의 글 양식을 준수하여 주세요 거래 지역...,가격이 궁금해요..^^\n얼마인가요ㅡ\n,제사모 중고장터,130,2022.03.16.,2022.03,제사모,2022,P,P
1,렌트카 알아보고 있는데 lpg 차량이 저렴한 이유가 뭘까요?,육지에서 이번주 토욜부터 제주 한달살이 시작하면서 렌트카 알아보고 있어요 육지에선 ...,요즘 전기차가 대세기도 하지만 가스값 장난아니게 올랏어요...또한 한달살기라면 자차...,Q. 종합 질문방,161,2022.03.16.,2022.03,제사모,2022,N,N


In [149]:
del df_cafe['lr긍부정이진']
df_cafe.rename(columns={'lr긍부정':'평가'},inplace=True)

In [159]:
df_cafe.내용[0]

' 보다 투명하고 빠른 거래를 위하여 아래의 글 양식을 준수하여 주세요   거래 지역  한림 거래 방법  직거래 연락 방법  연락처 안심번호 쪽지 채팅 가능  카카오톡 금지  판매 가격  연락주세요 판매 제품  봉고더블캡 제품 상태  중고차량사용감 있음 거의 새 것 미개봉  제품설명 및 사진 봉고더블캡  년 월식 정확히 만탔구요오토 차량 잡소리 하나도 안나요전기차 나와서 팔아요블랙박스후방감지 네비 열선시트 열선 핸들 만든공구함 박스 핸즈프리 '

In [166]:
df_cafe_commet.reset_index(inplace=True)
df_cafe.reset_index(inplace=True)

In [170]:
del df_cafe_commet['index']

In [174]:
# del df_cafe_commet['내용']
df_cafe_commet.rename(columns={'token':'내용'},inplace=True)

In [175]:
df_cafe_commet.head(2)

,제목,작성날짜,카페명,평가,월별,년별,내용
0,봉고3더블캡,2022.03.16.,제사모,E,2022.03,2022,가격 궁금하다 얼마 인가요
1,렌트카 알아보고 있는데 lpg 차량이 저렴한 이유가 뭘까요?,2022.03.16.,제사모,P,2022.03,2022,요즘 전기차 대세 기도 하지만 가스 값 장난 아니다 올 랏 어 요 또한 한 달 살기...


In [182]:
a= pd.concat([df_cafe,df_cafe_commet],axis=0)

In [184]:
del a['index']

In [186]:
a.reset_index(inplace=True)

In [189]:
df_cafe_commet_merge = copy.deepcopy(a)

In [191]:
df_cafe_commet_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10762 entries, 0 to 10761
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   제목      10762 non-null  object 
 1   내용      10762 non-null  object 
 2   유형      5683 non-null   object 
 3   조회수     5683 non-null   object 
 4   작성날짜    10762 non-null  object 
 5   월별      10762 non-null  float64
 6   카페명     10762 non-null  object 
 7   년별      10762 non-null  int64  
 8   평가      10762 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 756.8+ KB


In [192]:
df_cafe_commet_merge.to_csv('df_cafe_commet_merge.csv',index=False)

In [199]:
df_cafe_commet_merge.head(2)

,제목,내용,유형,조회수,작성날짜,월별,카페명,년별,평가
0,봉고3더블캡,보다 투명하고 빠른 거래를 위하여 아래의 글 양식을 준수하여 주세요 거래 지역...,제사모 중고장터,130,2022.03.16.,2022.03,제사모,2022,P
1,렌트카 알아보고 있는데 lpg 차량이 저렴한 이유가 뭘까요?,육지에서 이번주 토욜부터 제주 한달살이 시작하면서 렌트카 알아보고 있어요 육지에선 ...,Q. 종합 질문방,161,2022.03.16.,2022.03,제사모,2022,N


In [198]:
df_blog.head(2)

,제목,날짜,내용,긍부정,href,year,lr긍부정,lr긍부정이진분류,감성사전긍부정,전기차문장,전기차본문감성분석
0,2022년 4박 5일 신혼여행 후기 ing 제주도 (Feat. 일오반식당...,2022.03.17,제주도신혼여행후기내돈내산년 년 월 일 드뎌 결혼식이 끝나고 우도 가자마자 전기차를 ...,0,https://blog.naver.com/apple_378?Redirect=Log&...,2022,E,P,E,​ 오랜만에 바다를 보니 뻥 뚫리고 좋았어여 ㅎㅎ 같이 커플 트레이닝복으로 맞추고...,P
1,"제주공항숙소 시리우스 제주호텔: 트윈룸, 디럭스룸 장단점",2022.03.17,디럭스룸 후기 제주특별자치도 제주시 도령로 전화 공항의 지하 주차장은 지하층까지...,-1,https://blog.naver.com/zoonee96?Redirect=Log&l...,2022,E,P,F,​ 공항의 지하 주차장은 지하4층까지 있고 지하 1층에 전기차 충전소가 있어요 주차...,N


In [ ]:
del df_blog['내용','긍부정','href','lr긍부정','lr긍부정이진분류','감성사전긍부정']

In [ ]:
df_blog = pd.DataFrame(
    { "제목"
        
    }
)